In [32]:
import pandas as pd

In [33]:
df = pd.read_csv('ledwich_recfluence/channel_review.csv')
df.head()

,CHANNEL_TITLE,TAGS,CHANNEL_ID,LR,RELEVANCE,REVIEWER_LR,REVIEWER_TAGS,REVIEWER_MAIN_CHANNEL_ID,REVIEWER_RELEVANCE,REVIEWER_CODE,UPDATED,PUBLIC_REVIEWER_NOTES,PUBLIC_CREATOR_NOTES
0,Brave,NaN,UCFNTTISby1c_H-rm5Ww5rZg,C,0.0,C,NaN,NaN,0.0,mar,2021-12-22T01:00:52.5691836Z,NaN,NaN
1,GBNews,AntiSJW|Mainstream News|PartisanRight,UC0vn8ISa4LKMunLbzaXLnOQ,R,1.0,R,AntiSJW|Mainstream News|PartisanRight,NaN,1.0,mar,2021-10-25T21:17:49.7200458Z,NaN,NaN
2,The Black Conservative Preacher,AntiSJW|PartisanRight|QAnon,UC_QYq3wFd46rcaoKOvPHFgA,R,1.0,R,PartisanRight|QAnon|AntiSJW,NaN,1.0,mar,2021-06-08T01:27:45.4527338Z,This is the channel that had the senator who l...,NaN
3,Energy and Commerce Committee,StateFunded,UCCbD3bkHRcwiBsaL1lWE_QQ,C,1.0,C,StateFunded,NaN,1.0,mar,2021-03-26T02:15:09.3585105Z,NaN,NaN
4,That Star Wars Girl,AntiSJW,UCRWsFMLzGGJKLnAXJ9U93_A,R,0.8,R,AntiSJW,NaN,0.8,mar,2021-03-16T01:24:18.9299734Z,NaN,NaN


In [34]:
# Taking an extreme approach here, i.e. map to most extreme category first
# The fact that there aren't actually any 'MainstreamNews(K)' categories tells me that's their methodology.
# i.e. if it's 'Mainstream News' THEN we place a left or right label on it, depending on partisanship.
# I'll need to clarify with Hosseinmardi correspondents
ledwich_map = [
    ('FL', {'Socialist'}),
    ('FR', {'WhiteIdentitarian', 'QAnon', 'Conspiracy'}),
    ('AW', {'AntiSJW', 'MRA'}),
    ('L', {'Left', 'PartisanLeft',
           'SocialJustice'}),
    ('R', {'Right', 'PartisanRight',
           'ReligiousConservative', 'Libertarian(R)'}),
    ('C', {'Center'})
]

def map_labels(tags, leaning, label_map):
    if pd.isna(tags):
        return None
    tags = set(tags.split("|"))
    for mapping in label_map:
        new_label, old_labels = mapping
        if len(old_labels.intersection(tags))>0:
            return new_label
    if leaning:
        return leaning
    return None

In [35]:
new_labels = df.apply(
    lambda row: map_labels(row['TAGS'], row['LR'], ledwich_map), 
    axis=1
)

In [36]:
df['H_TAG'] = new_labels

In [37]:
df.head()

,CHANNEL_TITLE,TAGS,CHANNEL_ID,LR,RELEVANCE,REVIEWER_LR,REVIEWER_TAGS,REVIEWER_MAIN_CHANNEL_ID,REVIEWER_RELEVANCE,REVIEWER_CODE,UPDATED,PUBLIC_REVIEWER_NOTES,PUBLIC_CREATOR_NOTES,H_TAG
0,Brave,NaN,UCFNTTISby1c_H-rm5Ww5rZg,C,0.0,C,NaN,NaN,0.0,mar,2021-12-22T01:00:52.5691836Z,NaN,NaN,None
1,GBNews,AntiSJW|Mainstream News|PartisanRight,UC0vn8ISa4LKMunLbzaXLnOQ,R,1.0,R,AntiSJW|Mainstream News|PartisanRight,NaN,1.0,mar,2021-10-25T21:17:49.7200458Z,NaN,NaN,AW
2,The Black Conservative Preacher,AntiSJW|PartisanRight|QAnon,UC_QYq3wFd46rcaoKOvPHFgA,R,1.0,R,PartisanRight|QAnon|AntiSJW,NaN,1.0,mar,2021-06-08T01:27:45.4527338Z,This is the channel that had the senator who l...,NaN,FR
3,Energy and Commerce Committee,StateFunded,UCCbD3bkHRcwiBsaL1lWE_QQ,C,1.0,C,StateFunded,NaN,1.0,mar,2021-03-26T02:15:09.3585105Z,NaN,NaN,C
4,That Star Wars Girl,AntiSJW,UCRWsFMLzGGJKLnAXJ9U93_A,R,0.8,R,AntiSJW,NaN,0.8,mar,2021-03-16T01:24:18.9299734Z,NaN,NaN,AW


In [38]:
# Remove bitchute channels
def filter_bitchute(row):
    return len(row['CHANNEL_ID']) > 2 and row['CHANNEL_ID'][:2]=='UC'
bit = df.apply(filter_bitchute, axis=1)
df['BITCHUTE'] = bit
df = df[df['BITCHUTE']==True]
df.head()

,CHANNEL_TITLE,TAGS,CHANNEL_ID,LR,RELEVANCE,REVIEWER_LR,REVIEWER_TAGS,REVIEWER_MAIN_CHANNEL_ID,REVIEWER_RELEVANCE,REVIEWER_CODE,UPDATED,PUBLIC_REVIEWER_NOTES,PUBLIC_CREATOR_NOTES,H_TAG,BITCHUTE
0,Brave,NaN,UCFNTTISby1c_H-rm5Ww5rZg,C,0.0,C,NaN,NaN,0.0,mar,2021-12-22T01:00:52.5691836Z,NaN,NaN,None,True
1,GBNews,AntiSJW|Mainstream News|PartisanRight,UC0vn8ISa4LKMunLbzaXLnOQ,R,1.0,R,AntiSJW|Mainstream News|PartisanRight,NaN,1.0,mar,2021-10-25T21:17:49.7200458Z,NaN,NaN,AW,True
2,The Black Conservative Preacher,AntiSJW|PartisanRight|QAnon,UC_QYq3wFd46rcaoKOvPHFgA,R,1.0,R,PartisanRight|QAnon|AntiSJW,NaN,1.0,mar,2021-06-08T01:27:45.4527338Z,This is the channel that had the senator who l...,NaN,FR,True
3,Energy and Commerce Committee,StateFunded,UCCbD3bkHRcwiBsaL1lWE_QQ,C,1.0,C,StateFunded,NaN,1.0,mar,2021-03-26T02:15:09.3585105Z,NaN,NaN,C,True
4,That Star Wars Girl,AntiSJW,UCRWsFMLzGGJKLnAXJ9U93_A,R,0.8,R,AntiSJW,NaN,0.8,mar,2021-03-16T01:24:18.9299734Z,NaN,NaN,AW,True


In [39]:
df_ledwich = df[['CHANNEL_TITLE', 'CHANNEL_ID', 'H_TAG']]
df_ledwich = df_ledwich.drop_duplicates()
df_ledwich = df_ledwich[pd.isna(df_ledwich['H_TAG'])==False]

In [40]:
df_ledwich.head()

,CHANNEL_TITLE,CHANNEL_ID,H_TAG
1,GBNews,UC0vn8ISa4LKMunLbzaXLnOQ,AW
2,The Black Conservative Preacher,UC_QYq3wFd46rcaoKOvPHFgA,FR
3,Energy and Commerce Committee,UCCbD3bkHRcwiBsaL1lWE_QQ,C
4,That Star Wars Girl,UCRWsFMLzGGJKLnAXJ9U93_A,AW
5,Justin Murphy,UCfrlXVXz-pT9AmmDELhpyOw,AW


In [41]:
f = 'ribeiro_radicalization_pathways/'
ribeiro_map = {
    'L': ['mainstream/left.csv', 'mainstream/left-center.csv'],
    'C': ['mainstream/center.csv'],
    'AW': ['idw.csv'],
    'R': ['mainstream/right-center.csv', 'mainstream/right.csv', 'alt-lite.csv'],
    'FR': ['alt-right.csv']
}

In [42]:
df_ribeiro = pd.DataFrame(columns=['CHANNEL_TITLE', 'H_TAG'])


In [43]:
for category, lists in ribeiro_map.items():
    for file in lists:
        df_channel_list = pd.read_csv(f+file, names=['CHANNEL_TITLE'])
        num_rows = df_channel_list.shape[0]
        labels = [category for _ in range(num_rows)]
        df_channel_list['H_TAG'] = labels
        df_ribeiro = pd.concat([df_ribeiro, df_channel_list])

In [44]:
df_ribeiro

,CHANNEL_TITLE,H_TAG
0,cosmopolitan,L
1,democracy now,L
2,elite daily,L
3,good magazine,L
4,gq magazine,L
...,...,...
83,TRS Radio,FR
84,The Leftovers,FR
85,The Lion,FR
86,The Revolutionary Conservative,FR


In [45]:
df_final = pd.concat([df_ledwich, df_ribeiro])
df_final.head()

,CHANNEL_TITLE,CHANNEL_ID,H_TAG
1,GBNews,UC0vn8ISa4LKMunLbzaXLnOQ,AW
2,The Black Conservative Preacher,UC_QYq3wFd46rcaoKOvPHFgA,FR
3,Energy and Commerce Committee,UCCbD3bkHRcwiBsaL1lWE_QQ,C
4,That Star Wars Girl,UCRWsFMLzGGJKLnAXJ9U93_A,AW
5,Justin Murphy,UCfrlXVXz-pT9AmmDELhpyOw,AW


In [46]:
df_final.to_csv('h_labels.csv', index=False)
